## Methos: 基于蛋白结构的反应预测
> 2024-12-20

### 1. 导入必要的包

In [1]:
import pandas as pd
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
from tqdm import tqdm
from tools import btools
from tools import filetool 
import rxnrecer as production
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
import subprocess

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### 2. Load test data

In [2]:
def load_10folds_data(type='test'):
    if type == 'test':
        file_path = [f'{cfg.DIR_DATASET}validation/fold{fold_num+1}/valid.feather' for fold_num in range(10)]
        
    if type =='train':
        file_path = [f'{cfg.DIR_DATASET}validation/fold{fold_num+1}/train.feather' for fold_num in range(10)]
        
        
    res = [pd.read_feather(path)[['uniprot_id','seq','reaction_id']].rename(columns={'reaction_id': 'rxn_groundtruth'}) for path in file_path]
        
    return res


print('Loading pdbfile ...' )
data_pdb = pd.read_feather(cfg.FILE_DS_PDB_LIST)


print('Loading 10-folds  data ...' )
data_trian = load_10folds_data(type='train')
data_test = load_10folds_data(type='test')


Loading pdbfile ...
Loading 10-folds  data ...


### 3. Load Feaatures

In [7]:
tdi_embedding = pd.read_feather(cfg.FILE_DS_3DI_EMBEDDING)

In [7]:
trains_ids = [item.uniprot_id.to_list() for item in data_trian]

In [11]:
data_pdb[data_pdb.uniprot_id.isin(trains_ids[0]).reset_index(drop=True)]

,uniprot_id,seq,path_pdb,pdb_exist,pdb_check_ok
0,Q6GZX4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
1,Q6GZX3,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
2,Q197F8,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
4,Q6GZX2,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
5,Q6GZX1,MNAKYDTDQGVGRMLFLGTIGLAVVVGGLMAYGYYYDGKTPSSGTS...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
...,...,...,...,...,...
508580,B2MW50,MGLRYSKAVKDKYGDREIEGRATMTLNLPQGLYGRFNCKRCWFATK...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
508581,Q6RSS3,MGLRYSKEVRERHGDKDLEGRVPMTLNLPQGLYGRFNCKSCWFANR...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
508582,Q6UY62,MGNSKSKSKLSANQYEQQTVNSTKQVAILKRQAEPSLYGRHNCRCC...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True
508584,Q88470,MGNCNRTQKPSSSSNNLEKPPQAAEFRRTAEPSLYGRYNCKCCWFA...,/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data...,True,True


### 3. Load results
